In [ ]:
include("../load.jl")
using Plots
using StatsBase
function refine_thresholds(gm::GlobalModel, bbr::BlackBoxRegressor)
    if length(bbr.active_trees) == 1
        best_lower = getvalue(bbr.dependent_var)
        best_upper = bbr(solution(gm))[1]
        learn_constraint!(bbr, threshold = "upper" => best_upper)
        update_tree_constraints!(gm, bbr)
        learn_constraint!(bbr, threshold = "lower" =>best_lower)
        update_tree_constraints!(gm, bbr)
        return
    elseif length(bbr.active_trees) == 2
        bds = Dict(collect(values(bbr.active_trees))) # TODO: have a cleaner system for this.
        old_lower = bds["lower"]
        old_upper = bds["upper"]
        new_lower = getvalue(bbr.dependent_var)
        new_upper = bbr(solution(gm))[1]
        # Updating upper bounds
        if new_upper <= old_upper
            learn_constraint!(bbr, threshold = "upper" => new_upper)
            update_tree_constraints!(gm, bbr)
        else
            learn_constraint!(bbr, threshold = "upper" => old_upper) #TODO add warmstarts here. 
            update_tree_constraints!(gm, bbr)
        end
        # Updating lower bounds
        learn_constraint!(bbr, threshold = "lower" => (maximum([old_lower, new_lower]) + minimum([new_upper, old_upper]))/2)
        update_tree_constraints!(gm, bbr)
        return
        # if new_lower >= old_lower
        #     learn_constraint!(bbr, threshold = "lower" => (new_lower + minimum([new_upper, old_upper])/2)
        #     update_tree_constraints!(gm, bbr)
        #     return 
        # else
        #     learn_constraint!(bbr, # binary reduce the lower bound
        #         threshold = "lower" => (new_lower + old_lower)/2)
        #     update_tree_constraints!(gm, bbr)
        #     return
        # return 
        # end
    else 
        throw(OCTException("Cannot refine $(bbr.name) thresholds without having solved " *
                           "GlobalModel $(gm.name) with valid approximations first." ))
    end
end

In [ ]:
m = JuMP.Model(with_optimizer(CPLEX_SILENT))
@variable(m, -1 <= x <= 1)
@variable(m, y)
@constraint(m, 2*x + y >= 0)
@objective(m, Min, y)
@constraint(m, y >= (x+0.2)^2 - 1)
optimize!(m)
println(solution(m))

In [ ]:
# Plotting the feasible set
using Plots
X = -1:0.01:1
f(x) = (x+0.2)^2 - 1
g(x) = -2x
# the_max = maximum(f(-1:1))

plot(X, f, lw = 3)
# plot!(X, g, lw = 3)
scatter!([0.349], [-0.698], color=:red, size = (600,600))


In [ ]:
m = JuMP.Model(with_optimizer(CPLEX_SILENT))
@variable(m, -1 <= x <= 1)
@variable(m, y)
@constraint(m, 2*x + y >= 0)
@objective(m, Min, y)
gm = GlobalModel(model = m)
add_nonlinear_constraint(gm, :(x -> (x+0.2)^2 - 1), dependent_var = y)
bbr = gm.bbls[1]
set_param(bbr, :n_samples, 50)
uniform_sample_and_eval!(gm)

In [ ]:
# Suppose we did something really naive...
learn_constraint!(bbr, threshold = "upper" => quantile(bbr.Y, 0.9))
update_tree_constraints!(gm, bbr)
learn_constraint!(bbr, threshold = "lower" => quantile(bbr.Y, 0.01))
update_tree_constraints!(gm, bbr)
bbr.thresholds

In [ ]:
bbr.learners[end-1]

In [ ]:
# I optimize, and find an underestimator of the real solution 
optimize!(gm)
println(solution(gm), "  Real y at x: ", bbr(solution(gm))[1])
# So I narrow the search, and resample
leaf_sample(bbr)
refine_thresholds(gm, bbr)

In [ ]:
println("Progressively tightening thresholds...")
to_print = reshape(collect(values(bbr.thresholds)), (2, Int(length(bbr.thresholds)/2)))
println("Upper bounds: ", to_print[1,:])
println("Lower bounds: ", to_print[2,:])


In [ ]:
learn_constraint!(bbr, threshold = "lower" => -0.82)

In [ ]:
bbr.learners[end]